Here we present an implementation of the `time evolving block decimation` (TEBD) algorithm, which implements real or imaginary time evolution of a matrix product state (MPS). \
This method is based on a suzuki-trotter decomposition of a local Hamiltonian. 

- Computational cost: O(d χ^3)
- Makes use of the canonical form for MPS 

## Section 1: The infinite MPS ansatz

- The TEBD algorithm makes use of an infinite MPS, which describes a quantum state on an infinite 1D lattice. \
- We use an MPS with a 2-site unit cell, composed of `A` and `B` tensors, as this is the minimum unit cell compatible with TEBD. An MPS with a larger unit cell could be used if the periodicity of the quantum state that we are trying to describe is greater than 2 sites.
- For convenience we allow weights `sBA` and `sAB`, which are restricted to be diagonal matrices, to be positioned on links between the MPS tensors. 
- By convention, the indices on MPS tensors are ordered `left-centre-right`.

- we call the local dimension as `d` and the MPS bond dimension as `𝛘`

Fig.1
<img src="tebd_1.webp" width=500>

In [25]:
import numpy as np 
from numpy import linalg as LA
from ncon import ncon 
from scipy.linalg import expm
from scipy.sparse.linalg import eigs, LinearOperator

In [2]:
def IsingTF(J,h,δ):
    X = np.array([[0,1],[1,0]])
    Z = np.array([[1,0],[0,-1]])
    Id = np.eye(2)
    Hx = 0.5*(ncon([X,Id],[[-1,-3],[-2,-4]]) + ncon([Id,X],[[-1,-3],[-2,-4]]))
    Hz = ncon([Z,Z],[[-1,-3],[-2,-4]])
    H = -J*Hz -h*Hx
    Ug = expm(-δ*np.reshape(H,[4,4]))
    return H,Ug

In [3]:
def TensorCreation(d,D):
    A = np.random.rand(D,d,D);
    B = np.random.rand(D,d,D);
    sAB = np.ones(D) / np.sqrt(D);
    sBA = np.ones(D) / np.sqrt(D);
    return A,B,sAB,sBA

In [6]:
d = 2;
D = 5;
J = 1.0;
h = 0.3;
δ = 0.1;
A,B,sAB,sBA = TensorCreation(d,D);
H, Ug = IsingTF(J,h,δ);

## Section 2: Contraction of an infinite MPS

- Local expectation values from the network,\
$$<Ψ|oAB|Ψ>$$

Fig.2(i). 

- This is complicated in the infinite MPS, as we cannot directly contract an infinite network. 
- In practice, this difficulty can circumvented through the introduction of\
environment tensors `σ` and `μ`, which should be chosen to effectively account for the semi-infinite networks to the left and right of the region of interest.

Fig.2
<img src="tebd_2.webp" width=500>

- The environment tensors `σ` and `μ` should be chosen as the dominant fixed points of the contractions shown in Fig.3
- They are known as the (left/right) `MPS transfer operators`. 
- In practice, a simple method to find the environment tensors is to first initialize random tensors, and to then apply the `transfer operators` iteratively until they are converged to within a desired tolerance.

Fig.3
<img src="tebd_3.webp" width=500>

In [13]:
def left_contract(sigBA,A,B,sAB,sBA):
    D_BA = A.shape[0]
    if sigBA.shape[0] == D_BA:
        v0 = np.reshape(sigBA,(np.prod(sigBA.shape)))
    else:
        v0 = np.reshape(np.eye(D_BA) / D_BA, D_BA**2)
    # define network for transfer operator contract
    tensors = [np.diag(sBA), np.diag(sBA),A,A.conj(),np.diag(sAB),np.diag(sAB),B,B.conj()]
    connect = [[1,2],[1,3],[2,4],[3,5,6],[4,5,7],[6,8],[7,9],[8,10,-1],[9,10,-2]]
    # define function for boundary contraction and pass to eigs
    def left_iter(sigBA):
        return np.reshape(ncon([np.reshape(sigBA,[D_BA,D_BA]), *tensors],connect),
                          [D_BA**2,1])
    Dtemp, sigBA = eigs(LinearOperator((D_BA**2,D_BA**2), matvec=left_iter), 
                       k=1,which="LM",v0=v0,tol=1e-10)
    # normalize the environment density matrix sigBA
    if np.isrealobj(A):
        sigBA = np.real(sigBA)
    sigBA = np.reshape(sigBA,(D_BA,D_BA))
    sigBA = 0.5*(sigBA + np.conj(sigBA.T))
    sigBA = sigBA / np.trace(sigBA)
    # compute density matrix sigAB for A-B link
    sigAB = ncon([sigBA,np.diag(sBA),np.diag(sBA),A,A.conj()],
                 [[1,2],[1,3],[2,4],[3,5,-1],[4,5,-2]])
    sigAB = sigAB / np.trace(sigAB)
    return sigBA,sigAB

In [ ]:
def right_contract(muAB,A,B,sAB,sBA):
    D_AB = A.shape[2]
    if muAB.shape[0] == D_AB:
        v0 = np.reshape(muAB,np.prod(muAB.shape))
    else:
        v0 = np.reshape(np.eye(D_AB) / D_AB, D_AB**2)
     # define network for transfer operator contract
    tensors = [np.diag(sAB),np.diag(sAB),A,A.conj(),np.diag(sBA),np.diag(sBA),B,B.conj()]
#     connect = [[1,2],[1,3],[2,4],[3,5,6],[4,5,7],[6,8],[7,9],[8,10,-1],[9,10,-2]]
    connect = [[1,2],[3,1],[4,2],[6,5,3],[7,5,4],[8,6],[9,7],[-1,10,8],[-2,10,9]]
    # define function for boundary contraction and pass to eigs
    def right_iter(muAB):
        return ncon([np.reshape(muAB,[D_AB,D_AB]), *tensors], connect).reshape([D_AB**2,1])
    
    Dtemp, muAB = eigs(LinearOperator((D_AB**2, D_AB**2),matvec=right_iter),
                       k=1,which="LM",v0=v0,tol=1e-10)
    # normalize the environment density matrix muAB
    if np.isrealobj(A):
        muAB = np.real(muAB)
    muAB = np.reshape(muAB,[D_AB,D_AB])
    muAB = 0.5*(muAB + np.conj(muAB.T))
    muAB = muAB / np.trace(muAB)
    # compute density matrix muBA for B-A link
    muBA = ncon([muAB, np.diag(sAB),np.diag(sAB),A,A.conj()],
                [[1,2],[3,1],[5,2],[-1,4,3],[-2,4,5]])
    muBA = muBA / np.trace(muBA)
    return muAB, muBA

## Section 3: Canonical Form

## Section 4: MPS time evolution

We now describe the most important part of the TEBD algorithm:\
applying the time-evolution gates to the MPS. For simplicity we shall focus on the case of an imaginary-time evolution, although the same approach is used for a real-time evolution.\
Starting from an initial MPS |Ψ> we wish to construct the MPS for the time evolved state exp(-tH)|Ψ>. 

Fig.4
<img src="tebd_4.webp" width=500>

In [1]:
# define a Hamiltonian, here XX model

X = np.array([[0,1],[1,0]])
Y = np.array([[0, -1j],[1j, 0]])
hamAB = 0.5*(np.real(np.kron(X,X) + np.kron(Y,Y))).reshape(2,2,2,2)
hamBA = hamAB

# exponentiate Hamiltonian
tau = 0.1  # set time-step

gAB = expm(-tau * hamAB.reshape(d**2,d**2)).reshape(d,d,d,d)
gBA = expm(-tau * hamBA.reshape(d**2,d**2)).reshape(d,d,d,d)

NameError: name 'np' is not defined

A key step in TEBD is \
1. Contraction of a gate into a pair of MPS tensors
2. Then splitting back into a new pair of MPS tensors

- The adjacent link weights sBA should be absorbed into the composite tensor before truncation. This ensures that the truncation will minimize the global error. \
- The splitting of the composite tensor is accomplished via a truncated SVD.\
after the splitting, the adjacent link weights sBA should be again factored out, which can be accomplished by applying their inverses.\
some care should be taken to ensure that link weights sBA are not __too small__, which can otherwise lead to numerical instability due to finite precision arithmetic.

<img src="tebd_5.webp" width=500>

In [ ]:
def Apply_Gate(gAB,A,B,sAB,sBA,D,tol=1e-7):
    # ensure singular values are above tolerance threshold
    sBA_trim = sBA*(sBA > tol) + tol*(sBA < tol)
    # contract gate into the MPS, then deompose composite tensor with SVD
    d = A.shape[1]
    D_BA = sBA_trim.shape[0]
    tensors = [np.diag(sBA_trim), A, np.diag(sAB),B, np.diag(sBA_trim), gAB]
    connect = [[-1,1],[1,5,2],[2,4],[4,6,3],[3,-4],[-2,-3,5,6]]
    nshape = [d*D_BA, d*D]
    U,S,V = LA.svd(ncon(tensors, connect).reshape(nshape),full_matrices=False)
    # truncate to reduced dimension
    Dtemp = min(D,len(S))
    U = U[:,range(Dtemp)].reshape(sBA_trim.shape[0],d*Dtemp)
    V = V[range(Dtemp),:].reshape(d*Dtemp, D_BA)
    # remove environment weights to form new MPS tensors A and B
    A = (np.diag(1./sBA_trim) @ U).reshape(sBA_trim.shape[0],d,Dtemp)
    B = (V @ np.diag(1./sBA_trim)).reshape(Dtemp,d,D_BA)
    # new weights
    sAB = S[range(Dtemp)] / LA.norm(S[range(Dtemp)])
    return A,B,sAB

In [ ]:
d = 2;
D = 5;
J = 1.0;
h = 0.3;
δ = 0.1;
A,B,sAB,sBA = TensorCreation(d,D);
H, Ug = IsingTF(J,h,δ);
# initialize environment matrices
sigBA = np.eye(A.shape[0]) / A.shape[0];
muAB = np.eye(A.shape[2]) / A.shape[2];

# Section 5: Putting everything together

- The TEBD algorithm begins with a randomly initialized MPS and then iterates many of the steps described above . Specifically, each iteration involves:

1. solve for left/right environment tensors (using a Lanzcos method).

2. bring MPS into canonical form.

3. apply time-evolution gates across A-B links, and then across B-A links.


__Notes__:

solving for the left/right environment tensors and bringing the MPS into canonical form is computationally expensive to do every iteration.\
In practice it is more efficient to perform multiple steps of applying the time evolution gates between solving for the left/right environment tensors.\
It is often most efficient to start with a larger time-step 𝜏 for the initial iterations, and then reduce to smaller time-steps at later iterations (to improve the final accuracy).  

# Not Me

# ITEBD, ISING

In [4]:
def DoItebd(A,B,sAB,sBA,H,Ug):
    d = A.shape[1]
    D = A.shape[0]
    Ug = expm(-δ*np.reshape(H,[d**2,d**2]))
    # initialize environment matrices
    sigBA = np.eye(D) / D;
    mu
    pass

In [ ]:


""" Contract infinite MPS from the left for environment tensor sigBA """

sigBA = np.random.rand(chi, chi)  # initialize random starting point
tol = 1e-10  # set convergence tolerance

# define the tensor network
tensors = [np.diag(sBA), np.diag(sBA), A, A.conj(), np.diag(sAB),
           np.diag(sAB), B, B.conj()]
labels = [[1,2],[1,3],[2,4],[3,5,6],[4,5,7],[6,8],[7,9],[8,10,-1],[9,10,-2]]

for k in range(1000):
  sigBA_new = ncon([sigBA, *tensors], labels)  # contract transfer operator
  sigBA_new = sigBA_new / np.trace(sigBA_new)  # normalize
  if LA.norm(sigBA - sigBA_new) < tol:  # check convergence
    print('success!')
    break
  else:
    print('iter: %d, diff: %e' % (k, LA.norm(sigBA - sigBA_new)))
    sigBA = sigBA_new


# Not Me

In [ ]:
def left_contract_MPS(sigBA, sBA, A, sAB, B):
  # initialize the starting vector
  chiBA = A.shape[0]
  if sigBA.shape[0] == chiBA:
    v0 = sigBA.reshape(np.prod(sigBA.shape))
  else:
    v0 = (np.eye(chiBA) / chiBA).reshape(chiBA**2)

  # define network for transfer operator contract
  tensors = [np.diag(sBA), np.diag(sBA), A, A.conj(), np.diag(sAB),
             np.diag(sAB), B, B.conj()]
  labels = [[1, 2], [1, 3], [2, 4], [3, 5, 6], [4, 5, 7], [6, 8], [7, 9],
            [8, 10, -1], [9, 10, -2]]

  # define function for boundary contraction and pass to eigs
  def left_iter(sigBA):
    return ncon([sigBA.reshape([chiBA, chiBA]), *tensors],
                labels).reshape([chiBA**2, 1])
  Dtemp, sigBA = eigs(LinearOperator((chiBA**2, chiBA**2), matvec=left_iter),
                      k=1, which='LM', v0=v0, tol=1e-10)

  # normalize the environment density matrix sigBA
  if np.isrealobj(A):
    sigBA = np.real(sigBA)
  sigBA = sigBA.reshape(chiBA, chiBA)
  sigBA = 0.5 * (sigBA + np.conj(sigBA.T))
  sigBA = sigBA / np.trace(sigBA)

  # compute density matric sigAB for A-B link
  sigAB = ncon([sigBA, np.diag(sBA), np.diag(sBA), A, np.conj(A)],
               [[1, 2], [1, 3], [2, 4], [3, 5, -1], [4, 5, -2]])
  sigAB = sigAB / np.trace(sigAB)

  return sigBA, sigAB

# Time evolution of MPS (function):

In [ ]:
  
# doTEBD.py
# ---------------------------------------------------------------------
# Implementation of time evolution (real or imaginary) for MPS with 2-site unit
# cell (A-B), based on TEBD algorithm.
#
# by Glen Evenbly (c) for www.tensors.net, (v1.2) - last modified 6/2019

import numpy as np
from numpy import linalg as LA
from scipy.linalg import expm
from scipy.sparse.linalg import LinearOperator, eigs
from ncon import ncon
from typing import Optional


def doTEBD(hamAB: np.ndarray,
           hamBA: np.ndarray,
           A: np.ndarray,
           B: np.ndarray,
           sAB: np.ndarray,
           sBA: np.ndarray,
           chi: int,
           tau: float,
           evotype: Optional[str] = 'imag',
           numiter: Optional[int] = 1000,
           midsteps: Optional[int] = 10,
           E0: Optional[float] = 0.0):
  """
  Implementation of time evolution (real or imaginary) for MPS with 2-site unit
  cell (A-B), based on TEBD algorithm.
  Args:
    hamAB: nearest neighbor Hamiltonian coupling for A-B sites.
    hamBA: nearest neighbor Hamiltonian coupling for B-A sites.
    A: MPS tensor for A-sites of lattice.
    B: MPS tensor for B-sites of lattice.
    sAB: vector of weights for A-B links.
    sBA: vector of weights for B-A links.
    chi: maximum bond dimension of MPS.
    tau: time-step of evolution.
    evotype: set real (evotype='real') or imaginary (evotype='imag') evolution.
    numiter: number of time-step iterations to take.
    midsteps: number of time-steps between re-orthogonalization of the MPS.
    E0: specify the ground energy (if known).
  Returns:
    np.ndarray: MPS tensor for A-sites;
    np.ndarray: MPS tensor for B-sites;
    np.ndarray: vector sAB of weights for A-B links.
    np.ndarray: vector sBA of weights for B-A links.
    np.ndarray: two-site reduced density matrix rhoAB for A-B sites
    np.ndarray: two-site reduced density matrix rhoAB for B-A sites
  """

  # exponentiate Hamiltonian
  d = A.shape[1]
  if evotype == "real":
    gateAB = expm(1j * tau * hamAB.reshape(d**2, d**2)).reshape(d, d, d, d)
    gateBA = expm(1j * tau * hamBA.reshape(d**2, d**2)).reshape(d, d, d, d)
  elif evotype == "imag":
    gateAB = expm(-tau * hamAB.reshape(d**2, d**2)).reshape(d, d, d, d)
    gateBA = expm(-tau * hamBA.reshape(d**2, d**2)).reshape(d, d, d, d)

  # initialize environment matrices
  sigBA = np.eye(A.shape[0]) / A.shape[0]
  muAB = np.eye(A.shape[2]) / A.shape[2]

  for k in range(numiter + 1):
    if np.mod(k, midsteps) == 0 or (k == numiter):
      """ Bring MPS to normal form """

      # contract MPS from left and right
      sigBA, sigAB = left_contract_MPS(sigBA, sBA, A, sAB, B)
      muAB, muBA = right_contract_MPS(muAB, sBA, A, sAB, B)

      # orthogonalise A-B and B-A links
      B, sBA, A = orthog_MPS(sigBA, muBA, B, sBA, A)
      A, sAB, B = orthog_MPS(sigAB, muAB, A, sAB, B)

      # normalize the MPS tensors
      A_norm = np.sqrt(ncon([np.diag(sBA**2), A, np.conj(A), np.diag(sAB**2)],
                            [[1, 3], [1, 4, 2], [3, 4, 5], [2, 5]]))
      A = A / A_norm
      B_norm = np.sqrt(ncon([np.diag(sAB**2), B, np.conj(B), np.diag(sBA**2)],
                            [[1, 3], [1, 4, 2], [3, 4, 5], [2, 5]]))
      B = B / B_norm

      """ Compute energy and display """

      # compute 2-site local reduced density matrices
      rhoAB, rhoBA = loc_density_MPS(A, sAB, B, sBA)

      # evaluate the energy
      energyAB = ncon([hamAB, rhoAB], [[1, 2, 3, 4], [1, 2, 3, 4]])
      energyBA = ncon([hamBA, rhoBA], [[1, 2, 3, 4], [1, 2, 3, 4]])
      energy = 0.5 * (energyAB + energyBA)

      chitemp = min(A.shape[0], B.shape[0])
      enDiff = energy - E0
      print('iteration: %d of %d, chi: %d, t-step: %f, energy: %f, '
            'energy error: %e' % (k, numiter, chitemp, tau, energy, enDiff))

    """ Do evolution of MPS through one time-step """
    if k < numiter:
      # apply gate to A-B link
      A, sAB, B = apply_gate_MPS(gateAB, A, sAB, B, sBA, chi)

      # apply gate to B-A link
      B, sBA, A = apply_gate_MPS(gateBA, B, sBA, A, sAB, chi)

  rhoAB, rhoBA = loc_density_MPS(A, sAB, B, sBA)
  return A, B, sAB, sBA, rhoAB, rhoBA


def left_contract_MPS(sigBA, sBA, A, sAB, B):
  """ Contract an infinite 2-site unit cell from the left for the environment
  density matrices sigBA (B-A link) and sigAB (A-B link)"""

  # initialize the starting vector
  chiBA = A.shape[0]
  if sigBA.shape[0] == chiBA:
    v0 = sigBA.reshape(np.prod(sigBA.shape))
  else:
    v0 = (np.eye(chiBA) / chiBA).reshape(chiBA**2)

  # define network for transfer operator contract
  tensors = [np.diag(sBA), np.diag(sBA), A, A.conj(), np.diag(sAB),
             np.diag(sAB), B, B.conj()]
  labels = [[1, 2], [1, 3], [2, 4], [3, 5, 6], [4, 5, 7], [6, 8], [7, 9],
            [8, 10, -1], [9, 10, -2]]

  # define function for boundary contraction and pass to eigs
  def left_iter(sigBA):
    return ncon([sigBA.reshape([chiBA, chiBA]), *tensors],
                labels).reshape([chiBA**2, 1])
  Dtemp, sigBA = eigs(LinearOperator((chiBA**2, chiBA**2), matvec=left_iter),
                      k=1, which='LM', v0=v0, tol=1e-10)

  # normalize the environment density matrix sigBA
  if np.isrealobj(A):
    sigBA = np.real(sigBA)
  sigBA = sigBA.reshape(chiBA, chiBA)
  sigBA = 0.5 * (sigBA + np.conj(sigBA.T))
  sigBA = sigBA / np.trace(sigBA)

  # compute density matric sigAB for A-B link
  sigAB = ncon([sigBA, np.diag(sBA), np.diag(sBA), A, np.conj(A)],
               [[1, 2], [1, 3], [2, 4], [3, 5, -1], [4, 5, -2]])
  sigAB = sigAB / np.trace(sigAB)

  return sigBA, sigAB


def right_contract_MPS(muAB, sBA, A, sAB, B):
  """ Contract an infinite 2-site unit cell from the right for the environment
  density matrices muAB (A-B link) and muBA (B-A link)"""

  # initialize the starting vector
  chiAB = A.shape[2]
  if muAB.shape[0] == chiAB:
    v0 = muAB.reshape(np.prod(muAB.shape))
  else:
    v0 = (np.eye(chiAB) / chiAB).reshape(chiAB**2)

  # define network for transfer operator contract
  tensors = [np.diag(sAB), np.diag(sAB), A, A.conj(), np.diag(sBA),
             np.diag(sBA), B, B.conj()]
  labels = [[1, 2], [3, 1], [5, 2], [6, 4, 3], [7, 4, 5], [8, 6], [10, 7],
            [-1, 9, 8], [-2, 9, 10]]

  # define function for boundary contraction and pass to eigs
  def right_iter(muAB):
    return ncon([muAB.reshape([chiAB, chiAB]), *tensors],
                labels).reshape([chiAB**2, 1])
  Dtemp, muAB = eigs(LinearOperator((chiAB**2, chiAB**2), matvec=right_iter),
                     k=1, which='LM', v0=v0, tol=1e-10)

  # normalize the environment density matrix muAB
  if np.isrealobj(A):
    muAB = np.real(muAB)
  muAB = muAB.reshape(chiAB, chiAB)
  muAB = 0.5 * (muAB + np.conj(muAB.T))
  muAB = muAB / np.trace(muAB)

  # compute density matrix muBA for B-A link
  muBA = ncon([muAB, np.diag(sAB), np.diag(sAB), A, A.conj()],
              [[1, 2], [3, 1], [5, 2], [-1, 4, 3], [-2, 4, 5]])
  muBA = muBA / np.trace(muBA)

  return muAB, muBA


def orthog_MPS(sigBA, muBA, B, sBA, A, dtol=1e-12):
  """ set the MPS gauge across B-A link to the canonical form """

  # diagonalize left environment matrix
  dtemp, utemp = LA.eigh(sigBA)
  chitemp = sum(dtemp > dtol)
  DL = dtemp[range(-1, -chitemp - 1, -1)]
  UL = utemp[:, range(-1, -chitemp - 1, -1)]

  # diagonalize right environment matrix
  dtemp, utemp = LA.eigh(muBA)
  chitemp = sum(dtemp > dtol)
  DR = dtemp[range(-1, -chitemp - 1, -1)]
  UR = utemp[:, range(-1, -chitemp - 1, -1)]

  # compute new weights for B-A link
  weighted_mat = (np.diag(np.sqrt(DL)) @ UL.T @ np.diag(sBA)
                  @ UR @ np.diag(np.sqrt(DR)))
  UBA, stemp, VhBA = LA.svd(weighted_mat, full_matrices=False)
  sBA = stemp / LA.norm(stemp)

  # build x,y gauge change matrices, implement gauge change on A and B
  x = np.conj(UL) @ np.diag(1 / np.sqrt(DL)) @ UBA
  y = np.conj(UR) @ np.diag(1 / np.sqrt(DR)) @ VhBA.T
  A = ncon([y, A], [[1, -1], [1, -2, -3]])
  B = ncon([B, x], [[-1, -2, 2], [2, -3]])

  return B, sBA, A


def apply_gate_MPS(gateAB, A, sAB, B, sBA, chi, stol=1e-7):
  """ apply a gate to an MPS across and a A-B link. Truncate the MPS back to
  some desired dimension chi"""

  # ensure singular values are above tolerance threshold
  sBA_trim = sBA * (sBA > stol) + stol * (sBA < stol)

  # contract gate into the MPS, then deompose composite tensor with SVD
  d = A.shape[1]
  chiBA = sBA_trim.shape[0]
  tensors = [np.diag(sBA_trim), A, np.diag(sAB), B, np.diag(sBA_trim), gateAB]
  connects = [[-1, 1], [1, 5, 2], [2, 4], [4, 6, 3], [3, -4], [-2, -3, 5, 6]]
  nshape = [d * chiBA, d * chiBA]
  utemp, stemp, vhtemp = LA.svd(ncon(tensors, connects).reshape(nshape),
                                full_matrices=False)

  # truncate to reduced dimension
  chitemp = min(chi, len(stemp))
  utemp = utemp[:, range(chitemp)].reshape(sBA_trim.shape[0], d * chitemp)
  vhtemp = vhtemp[range(chitemp), :].reshape(chitemp * d, chiBA)

  # remove environment weights to form new MPS tensors A and B
  A = (np.diag(1 / sBA_trim) @ utemp).reshape(sBA_trim.shape[0], d, chitemp)
  B = (vhtemp @ np.diag(1 / sBA_trim)).reshape(chitemp, d, chiBA)

  # new weights
  sAB = stemp[range(chitemp)] / LA.norm(stemp[range(chitemp)])

  return A, sAB, B


def loc_density_MPS(A, sAB, B, sBA):
  """ Compute the local reduced density matrices from an MPS (assumend to be
  in canonical form)."""

  # recast singular weights into a matrix
  mAB = np.diag(sAB)
  mBA = np.diag(sBA)

  # contract MPS for local reduced density matrix (A-B)
  tensors = [np.diag(sBA**2), A, A.conj(), mAB, mAB, B, B.conj(),
             np.diag(sBA**2)]
  connects = [[3, 4], [3, -3, 1], [4, -1, 2], [1, 7], [2, 8], [7, -4, 5],
              [8, -2, 6], [5, 6]]
  rhoAB = ncon(tensors, connects)

  # contract MPS for local reduced density matrix (B-A)
  tensors = [np.diag(sAB**2), B, B.conj(), mBA, mBA, A, A.conj(),
             np.diag(sAB**2)]
  connects = [[3, 4], [3, -3, 1], [4, -1, 2], [1, 7], [2, 8], [7, -4, 5],
              [8, -2, 6], [5, 6]]
  rhoBA = ncon(tensors, connects)

  return rhoAB, rhoBA
